## Using AutoKeras 1.0.16.post1 to solve [Digit Recognizer Kaggle Competition](https://www.kaggle.com/c/digit-recognizer/overview)

### Unzip and upload datasets

Do not use Tensorflow's built-in MNIST dataset. The train/test split ratios are different and will introduce data leak.

In [ ]:
import pandas as pd

train_raw = pd.read_csv('./train.csv')
test_raw = pd.read_csv('./test.csv')

train_raw

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Reshape to (28, 28) images as ndarray

In [ ]:
y_train = train_raw.pop('label').to_numpy()
x_train = train_raw.to_numpy().reshape(-1, 28, 28)
x_test = test_raw.to_numpy().reshape(-1, 28, 28)

### Train a CNN model

In [ ]:
import autokeras as ak
import tensorflow as tf

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ConvBlock()(output_node)
output_node = ak.SpatialReduction()(output_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(
    inputs=input_node, outputs=output_node,
    max_trials=100. overwrite=True)

clf.fit(
    x_train, y_train, 
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Trial 70 Complete [00h 00m 38s]
val_loss: 0.033836085349321365

Best val_loss So Far: 0.025965550914406776
Total elapsed time: 02h 10m 45s
INFO:tensorflow:Oracle triggered exit


Epoch 1/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.2403 - accuracy: 0.9234
Epoch 2/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0825 - accuracy: 0.9739
Epoch 3/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0642 - accuracy: 0.9800: 0s -
Epoch 4/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0546 - accuracy: 0.9828
Epoch 5/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0480 - accuracy: 0.9850
Epoch 6/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0423 - accuracy: 0.9870
Epoch 7/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0378 - accuracy: 0.9885
Epoch 8/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0375 - accuracy: 0.9880
Epoch 9/11
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0350 - accuracy: 0.9885
Epoch 10/11
1313/1313 [==============================] - 7s 5ms/step - loss:

### Predict test labels

In [ ]:
predicted = clf.predict(x_test).flatten().astype('uint8')

875/875 [==============================] - 2s 2ms/step


### Generate Kaggle submission

In [ ]:
import numpy as np
import pandas as pd

submission = pd.DataFrame({'ImageId': np.arange(x_test.shape[0])+1, 'Label': predicted})
submission.to_csv('./submission.csv', index=False)
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


* Kaggle score: 0.99153